In [21]:
#imports and scraping raw data from site
#df is the actual betting line
#df2 has other info, mostly using to get player id to name dict

import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import datetime as dt
import random
import re
from unidecode import unidecode


In [22]:
import urllib3
import json

import urllib
class AppURLopener(urllib.request.FancyURLopener):
    version = "Mozilla/5.0"

    
dog_props_url = 'https://api.underdogfantasy.com/beta/v3/over_under_lines'
opener = AppURLopener()
response = opener.open(dog_props_url)

print(response.status)

data = json.loads(response.read())

C:\Users\User\AppData\Local\Temp\ipykernel_5076\3263823425.py:10: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  opener = AppURLopener()


200


In [23]:
data.keys()

dict_keys(['appearances', 'games', 'over_under_lines', 'players', 'solo_games'])

In [24]:
df = pd.json_normalize(list(data.values()))
df.index = data.keys()
new = df.T['over_under_lines'].values
lines = pd.json_normalize(new)
lines.sample(2)

,id,live_event,live_event_stat,options,over_under_id,rank,stat_value,status,over_under.id,over_under.appearance_stat.id,over_under.appearance_stat.appearance_id,over_under.appearance_stat.display_stat,over_under.appearance_stat.graded_by,over_under.appearance_stat.stat,over_under.boost,over_under.scoring_type_id,over_under.title
609,fb88db02-01a6-4613-8d8e-270efc242da9,False,None,[{'id': '66208262-f90d-489b-9994-91fc43e72b10'...,3059a889-3f4c-4a99-948a-c0c108324fbf,8995,70.0,active,3059a889-3f4c-4a99-948a-c0c108324fbf,e42d18f6-7fbc-4b35-af7e-b97d5c88e404,1090df16-e10e-4de7-8d8f-23c13c533af5,Strokes,high_score,strokes,None,None,Antoine Rozner Strokes O/U
584,eb142269-a553-4a35-9660-2d5ffaacb04b,False,None,[{'id': '4fc3389b-5edf-4e54-9436-09cd12bb1d24'...,ffcb4230-4e95-4200-9489-579e6fc5273e,8008,2.0,active,ffcb4230-4e95-4200-9489-579e6fc5273e,2484e7bf-983e-4e64-9ddf-494d58609e59,b63bf45a-d79e-4f21-895a-cc536b3bee28,Bogeys or Worse,high_score,bogeys_or_worse,None,None,Sungjae Im Bogeys or Worse O/U


In [25]:
test =  pd.json_normalize(df.T['players'])
player_df = test.copy(deep=True)
player_df = player_df[['id','first_name', 'last_name',
       'sport_id','team_id']]
player_df = player_df.dropna()#

player_df.loc[player_df.sport_id=='ESPORTS']
x = player_df[player_df.sport_id=='ESPORTS']
p2esport = x[['last_name','first_name']].to_dict('split')
p2esport = dict(p2esport['data'])

In [26]:
test =  pd.json_normalize(df.T['players'])
player_df = test.copy(deep=True)
player_df = player_df[['id','first_name', 'last_name',
       'sport_id','team_id']]
player_df = player_df.dropna()
player_df.loc[player_df['sport_id'] != 'ESPORTS', 'full_name'] = player_df['first_name']+' ' + player_df['last_name']
player_df.loc[player_df['sport_id'] == 'ESPORTS', 'full_name'] = player_df['last_name']
#player_df['full_name'] = player_df['first_name']+' ' + player_df['last_name']
player_df = player_df[['full_name','sport_id','team_id']]
p2l = player_df[['full_name','sport_id']].to_dict('split')
p2tm = player_df[['full_name','team_id']].to_dict('split')
temp = p2l['data']
temp2 = p2tm['data']
p2l = dict(temp)
p2tm = dict(temp2)

In [27]:
player_df

,full_name,sport_id,team_id
0,Jimmy Butler,NBA,4ea17ce5-80e9-47b3-9955-2729a02bbe92
1,Bam Adebayo,NBA,4ea17ce5-80e9-47b3-9955-2729a02bbe92
2,Max Strus,NBA,4ea17ce5-80e9-47b3-9955-2729a02bbe92
3,Gabe Vincent,NBA,4ea17ce5-80e9-47b3-9955-2729a02bbe92
4,Caleb Martin,NBA,4ea17ce5-80e9-47b3-9955-2729a02bbe92
...,...,...,...
327,Knight~ (Smiling Knight),ESPORTS,2008b5a6-83c4-462c-b76f-70d1db2b46dc
328,SLATEM$,ESPORTS,2008b5a6-83c4-462c-b76f-70d1db2b46dc
329,ILICH-,ESPORTS,2008b5a6-83c4-462c-b76f-70d1db2b46dc
330,N1ght,ESPORTS,2008b5a6-83c4-462c-b76f-70d1db2b46dc


In [28]:
player_df.sport_id.unique()

array(['NBA', 'MLB', 'NHL', 'WNBA', 'ESPORTS'], dtype=object)

In [29]:
first2 = player_df['full_name'].apply(lambda x: ' '.join(x.split()[0:2]))
names = pd.DataFrame((first2,player_df.full_name))
names = names.T
names.columns=['first2','fullname']
names.loc[names.first2=='Jazz Chisholm']
first2full = names.to_dict('split')
first2full = dict(first2full['data'])

In [30]:
games = pd.json_normalize(df.T['games'].values).dropna(how='all')
#games = games.loc[games['sport_id']=='NBA']
games.tail(3)

,id,away_team_id,away_team_score,home_team_id,home_team_score,match_progress,period,scheduled_at,season_type,sport_id,status,title,type,year
30,39946.0,72638d19-df71-4400-ad1c-fa96f0fd102e,0.0,ae531b97-3116-4e25-b346-2ae599a693f5,0.0,NaN,0.0,2023-05-25T17:00:00Z,regular,ESPORTS,scheduled,Dota: Hokori vs. Balrogs,Game,2023.0
31,39947.0,6b0b1001-d17e-4777-8de5-c4d8adc8a475,0.0,ec521a60-2831-4d23-a2ac-39da29b13c58,0.0,NaN,0.0,2023-05-25T20:00:00Z,regular,ESPORTS,scheduled,Dota: Evil Geniuses vs. Infinity,Game,2023.0
32,39948.0,e0a3b297-6fd6-45bc-977e-9940b7140968,0.0,2008b5a6-83c4-462c-b76f-70d1db2b46dc,0.0,NaN,0.0,2023-05-25T23:00:00Z,regular,ESPORTS,scheduled,Dota: beastcoast vs. Thunder Awaken,Game,2023.0


In [31]:
def to_central(date_str):
    x = pd.to_datetime(date_str)
    x = x.tz_convert('US/Central')
    return str(x.date())
to_central('2023-05-20T00:40:00Z')

'2023-05-19'

In [32]:
games.scheduled_at = games.scheduled_at.apply(to_central)

In [33]:
#grabbing team names from match title
matchup = games['title']
home = []
away = []
for i in matchup:
    temp = re.split(' @ |, |vs.',i)
    home.append(temp[0])
    away.append(temp[1])
#fixing esports team names
for i in range(len(home)):
    home[i] = home[i].split(': ')[-1]
    
for i in range(len(away)):
    away[i] = away[i].split(': ')[-1]

matchup = pd.DataFrame((away,home,games.away_team_id,games.home_team_id)).T
matchup.columns = ['Home', 'Away','away_id','home_id']
dates = games['scheduled_at'].values
#temp = []
#for i in dates:
#    temp.append(i.split('T')[0])
matchup['Date'] = dates
id2tm =  matchup[['home_id','Home']].to_dict('split')
id2tmaway =  matchup[['away_id','Away']].to_dict('split')
id2tm = dict(id2tm['data'])
id2tmaway = dict(id2tmaway['data'])
id2tm.update(id2tmaway)

In [34]:
games

,id,away_team_id,away_team_score,home_team_id,home_team_score,match_progress,period,scheduled_at,season_type,sport_id,status,title,type,year
0,39941.0,4ea17ce5-80e9-47b3-9955-2729a02bbe92,0.0,8f5852b6-c8dc-4cce-980a-eab09b28919a,0.0,NaN,0.0,2023-05-25,post,NBA,scheduled,MIA @ BOS,Game,2022.0
1,31417.0,4d5e7fe3-d039-43fc-8084-b7f16219c9be,0.0,f8aa5fb3-bf30-4f1d-bc89-7dc208a9c5d3,0.0,NaN,0.0,2023-05-24,regular,MLB,scheduled,TOR @ TB,Game,2023.0
2,29823.0,e83a3a19-0890-46d9-967f-f7b29d7d9ca6,0.0,1748a711-2091-499b-9d33-dbd35ef8a62c,0.0,NaN,0.0,2023-05-24,regular,MLB,scheduled,STL @ CIN,Game,2023.0
3,30177.0,1fd765d9-7354-4926-bb4b-368d86d202b0,0.0,bbb3bae8-1013-4168-8f72-b75d55f0d7f6,0.0,NaN,0.0,2023-05-24,regular,MLB,scheduled,BAL @ NYY,Game,2023.0
4,31237.0,898ed118-c7ab-4e09-bff4-2a2a7af29053,0.0,30a238e0-4b7b-4bbb-a471-6b1b7a184a22,0.0,NaN,0.0,2023-05-24,regular,MLB,scheduled,SD @ WSH,Game,2023.0
5,31352.0,5a5a4e8a-d364-4fba-9b76-add9ed5d9ad7,0.0,02c19f09-f45a-4808-a128-e5bbf9d9577e,0.0,NaN,0.0,2023-05-24,regular,MLB,scheduled,LAD @ ATL,Game,2023.0
6,30298.0,1321328d-b5fa-44ad-b842-668ec37f0b4b,0.0,0a2c9da4-7227-4055-bf48-bb4bf5c8f410,0.0,NaN,0.0,2023-05-24,regular,MLB,scheduled,NYM @ CHC,Game,2023.0
7,30887.0,b921da74-fa62-477c-87b5-84885a6fe0e4,0.0,61cc4195-1a95-4f1a-acf4-44b806f937f5,0.0,NaN,0.0,2023-05-24,regular,MLB,scheduled,DET @ KC,Game,2023.0
8,31264.0,86655f8c-b7e8-4ed3-809c-d0d2cb0f911b,0.0,9be33c5c-6127-4d03-bbbc-e14900856b85,0.0,NaN,0.0,2023-05-24,regular,MLB,scheduled,MIA @ COL,Game,2023.0
9,31668.0,e0b9d617-58af-4537-a82c-a802ef2b969e,0.0,082ccc12-6c1f-4027-a135-fc88f237c6a1,0.0,NaN,0.0,2023-05-24,regular,MLB,scheduled,BOS @ LAA,Game,2023.0


In [35]:
id2tm

{'8f5852b6-c8dc-4cce-980a-eab09b28919a': 'BOS',
 'f8aa5fb3-bf30-4f1d-bc89-7dc208a9c5d3': 'TB',
 '1748a711-2091-499b-9d33-dbd35ef8a62c': 'CIN',
 'bbb3bae8-1013-4168-8f72-b75d55f0d7f6': 'NYY',
 '30a238e0-4b7b-4bbb-a471-6b1b7a184a22': 'WSH',
 '02c19f09-f45a-4808-a128-e5bbf9d9577e': 'ATL',
 '0a2c9da4-7227-4055-bf48-bb4bf5c8f410': 'CHC',
 '61cc4195-1a95-4f1a-acf4-44b806f937f5': 'KC',
 '9be33c5c-6127-4d03-bbbc-e14900856b85': 'COL',
 '082ccc12-6c1f-4027-a135-fc88f237c6a1': 'LAA',
 'fc7af4cb-e8c4-46f6-9e23-4a5df7914ba0': 'SEA',
 'e1a07af8-365d-4261-8b9b-3ccbc99e5e21': 'FLA',
 '7497705e-afa6-441e-913f-a6b2e7a99080': 'LA',
 'bd252434-828d-48d8-9884-6b0c7dd1b910': 'PHX',
 'f4ac5e20-ca56-4958-aacc-98e55aed3c88': ' Atlanta FaZe',
 '226fbf2f-8c3a-4d36-b752-34f38ae9c6f6': ' Toronto Ultra',
 '16f18a85-e19f-4943-bb8a-7eb36d8b5ae4': ' OpTic Texas',
 '4490ea6e-6410-4dce-9919-911f14f61186': ' Boston Breach',
 '6b0b1001-d17e-4777-8de5-c4d8adc8a475': 'Evil Geniuses ',
 '765d5670-ce67-4848-a081-b712f72b0cfc'

In [36]:
lines = lines[['stat_value','over_under.appearance_stat.stat', 'over_under.title']]
lines
players = lines['over_under.title'].values
guys = []
for i in players:
    temp = i.split()
    final = temp[0] + ' '+temp[1]
    guys.append(final)
    
guys
lines= lines.rename(columns={'stat_value':'Line','over_under.appearance_stat.stat':'Stat', 'over_under.title':'title'})
lines['Player']= guys
lines['Player'] = lines['Player'].apply(lambda x: unidecode(x).replace('_',' '))


In [37]:
def map_name(x):
    if x is None:
        return None
    if x in first2full:
        return first2full[x]
    elif x.split(': ')[-1] in first2full:
        return x.split(': ')[-1]
    elif x.split()[-1] in first2full:
        return x.split()[-1]
    else:
        return x

lines['Player'] = lines['Player'].apply(lambda x: map_name(x))
lines.Player.unique()

array(['Jimmy Butler', 'Bam Adebayo', 'Max Strus', 'Gabe Vincent',
       'Caleb Martin', 'Kevin Love', 'Jayson Tatum', 'Jaylen Brown',
       'Malcolm Brogdon', 'Derrick White', 'Marcus Smart', 'Al Horford',
       'Grant Williams', 'Robert Williams', 'Yusei Kikuchi',
       'Vladimir Guerrero Jr.', 'Bo Bichette', 'Whit Merrifield',
       'Nathan Lukes', 'Shane McClanahan', 'Wander Franco', 'Yandy Diaz',
       'Jose Siri', 'Steven Matz', 'Willson Contreras',
       'Paul Goldschmidt', 'Nolan Arenado', 'Lars Nootbaar',
       'Alec Burleson', 'Tommy Edman', 'Brendan Donovan', 'Ben Lively',
       'Spencer Steer', 'Jonathan India', 'Tyler Stephenson',
       'Kevin Newman', 'Jose Barrero', 'Tyler Wells', 'Adley Rutschman',
       'Ryan Mountcastle', 'Adam Frazier', 'Nestor Cortes',
       'Harrison Bader', 'Aaron Judge', 'Gleyber Torres',
       'Isiah Kiner-Falefa', 'Jake Bauers', 'Kyle Higashioka',
       'Ryan Weathers', 'Fernando Tatis Jr.', 'Juan Soto',
       'Xander Bogaerts', 

In [38]:
league = []
for p in guys:
    if p in p2l:
        league.append(p2l[p])
    elif p.split()[-1] in p2l:
        league.append(p2l[p.split()[-1]])
    else:
        league.append('other')
league[0:5]

['NBA', 'NBA', 'NBA', 'NBA', 'NBA']

In [39]:
guys[1000:1007]

[]

In [40]:
team_id = []
for p in guys:
    if p in p2l:
        team_id.append(p2tm[p])
    elif p.split()[-1] in p2l:
        team_id.append(p2tm[p.split()[-1]])
    else:
        team_id.append('other')
team_id[0:5]

['4ea17ce5-80e9-47b3-9955-2729a02bbe92',
 '4ea17ce5-80e9-47b3-9955-2729a02bbe92',
 '4ea17ce5-80e9-47b3-9955-2729a02bbe92',
 '4ea17ce5-80e9-47b3-9955-2729a02bbe92',
 '4ea17ce5-80e9-47b3-9955-2729a02bbe92']

In [41]:
lines['League'] = league
lines

,Line,Stat,title,Player,League
0,28.5,points,Jimmy Butler Points O/U,Jimmy Butler,NBA
1,7.0,rebounds,Jimmy Butler Rebounds O/U,Jimmy Butler,NBA
2,41.5,pts_rebs_asts,Jimmy Butler Pts + Rebs + Asts O/U,Jimmy Butler,NBA
3,6.0,assists,Jimmy Butler Assists O/U,Jimmy Butler,NBA
4,34.5,pts_asts,Jimmy Butler Points + Assists O/U,Jimmy Butler,NBA
...,...,...,...,...,...
607,7.0,strokes,Jordan L. Smith Strokes O/U,Jordan L.,other
608,70.0,strokes,Alexander Bjork Strokes O/U,Alexander Bjork,other
609,70.0,strokes,Antoine Rozner Strokes O/U,Antoine Rozner,other
610,70.5,strokes,Joost Luiten Strokes O/U,Joost Luiten,other


In [42]:
def map_team(x):
    if x in p2tm:
        return p2tm[x]
    elif p.split()[-1] in p2l:
        return p2tm[p.split()[-1]]
    else:
        return x

lines['team_id'] = lines.Player.apply(lambda x: map_team(x))
lines

,Line,Stat,title,Player,League,team_id
0,28.5,points,Jimmy Butler Points O/U,Jimmy Butler,NBA,4ea17ce5-80e9-47b3-9955-2729a02bbe92
1,7.0,rebounds,Jimmy Butler Rebounds O/U,Jimmy Butler,NBA,4ea17ce5-80e9-47b3-9955-2729a02bbe92
2,41.5,pts_rebs_asts,Jimmy Butler Pts + Rebs + Asts O/U,Jimmy Butler,NBA,4ea17ce5-80e9-47b3-9955-2729a02bbe92
3,6.0,assists,Jimmy Butler Assists O/U,Jimmy Butler,NBA,4ea17ce5-80e9-47b3-9955-2729a02bbe92
4,34.5,pts_asts,Jimmy Butler Points + Assists O/U,Jimmy Butler,NBA,4ea17ce5-80e9-47b3-9955-2729a02bbe92
...,...,...,...,...,...,...
607,7.0,strokes,Jordan L. Smith Strokes O/U,Jordan L.,other,Jordan L.
608,70.0,strokes,Alexander Bjork Strokes O/U,Alexander Bjork,other,Alexander Bjork
609,70.0,strokes,Antoine Rozner Strokes O/U,Antoine Rozner,other,Antoine Rozner
610,70.5,strokes,Joost Luiten Strokes O/U,Joost Luiten,other,Joost Luiten


In [43]:
def map_id(x):
    if x in id2tm:
        return id2tm[x]
    else:
        return x
#lines['team_id'] = lines.team_id.apply(lambda x: map_id(x))


In [44]:
final = pd.merge(lines,matchup, how='left',left_on='team_id',right_on='away_id')
final = final.drop(['Home', 'Away','home_id','away_id'], axis=1)
final2 = pd.merge(final,matchup, how='left',left_on='team_id',right_on='home_id')
final2 = final2.drop(['Home', 'Away','home_id','away_id'], axis=1)
final2['Date'] = final2['Date_x'].fillna(final2['Date_y'])
final2 = final2.drop(['Date_x', 'Date_y'], axis=1)
final2['team_id'] = final2.team_id.apply(lambda x: map_id(x))
final2#.loc[final.Player=='Jimmy Butler']

,Line,Stat,title,Player,League,team_id,Date
0,28.5,points,Jimmy Butler Points O/U,Jimmy Butler,NBA,MIA,2023-05-25
1,7.0,rebounds,Jimmy Butler Rebounds O/U,Jimmy Butler,NBA,MIA,2023-05-25
2,41.5,pts_rebs_asts,Jimmy Butler Pts + Rebs + Asts O/U,Jimmy Butler,NBA,MIA,2023-05-25
3,6.0,assists,Jimmy Butler Assists O/U,Jimmy Butler,NBA,MIA,2023-05-25
4,34.5,pts_asts,Jimmy Butler Points + Assists O/U,Jimmy Butler,NBA,MIA,2023-05-25
...,...,...,...,...,...,...,...
607,7.0,strokes,Jordan L. Smith Strokes O/U,Jordan L.,other,Jordan L.,NaN
608,70.0,strokes,Alexander Bjork Strokes O/U,Alexander Bjork,other,Alexander Bjork,NaN
609,70.0,strokes,Antoine Rozner Strokes O/U,Antoine Rozner,other,Antoine Rozner,NaN
610,70.5,strokes,Joost Luiten Strokes O/U,Joost Luiten,other,Joost Luiten,NaN


In [45]:
final2.columns = ['Line', 'Stat', 'title', 'Player', 'League', 'Team', 'Date']

In [46]:
df = final2[['Line', 'Stat', 'Player', 'League', 'Team', 'Date']].copy(deep=True)
df

,Line,Stat,Player,League,Team,Date
0,28.5,points,Jimmy Butler,NBA,MIA,2023-05-25
1,7.0,rebounds,Jimmy Butler,NBA,MIA,2023-05-25
2,41.5,pts_rebs_asts,Jimmy Butler,NBA,MIA,2023-05-25
3,6.0,assists,Jimmy Butler,NBA,MIA,2023-05-25
4,34.5,pts_asts,Jimmy Butler,NBA,MIA,2023-05-25
...,...,...,...,...,...,...
607,7.0,strokes,Jordan L.,other,Jordan L.,NaN
608,70.0,strokes,Alexander Bjork,other,Alexander Bjork,NaN
609,70.0,strokes,Antoine Rozner,other,Antoine Rozner,NaN
610,70.5,strokes,Joost Luiten,other,Joost Luiten,NaN


In [47]:
df = df.reset_index(drop=True)

In [48]:
df.Stat.unique()

array(['points', 'rebounds', 'pts_rebs_asts', 'assists', 'pts_asts',
       'rebs_asts', 'pts_rebs', 'blks_stls', 'turnovers', 'blocks',
       'steals', 'strikeouts', 'pitch_count', 'fantasy_points',
       'hits_runs_rbis', 'singles', 'total_bases', 'hits', 'walks',
       'runs', 'walks_allowed', 'rbis', 'goals_against', 'saves', 'shots',
       'kills_on_maps_1_2_3', 'map_1_kills', 'kills_on_map_3',
       'map_1_and_2_kills', 'headshots on maps 1+2', 'game_1_and_2_kills',
       'birdies_or_better', 'strokes', 'bogeys_or_worse'], dtype=object)

In [49]:
syntax = {
    'pts_rebs_asts': 'PTS+TRB+AST',
    'pts_rebs': 'PTS+TRB',
    'points': 'PTS',
    'free_throws_made': 'FT',
    'fantasy_points': 'Fantasy',
    'rebounds': 'TRB',
    'assists': 'AST',
    'steals': 'STL',
    'blocks': 'BLK',
    'blks_stls': 'BLK+STL',
    'turnovers': 'TO',
    'rebs_asts': 'TRB+AST',
    'pts_asts': 'PTS+AST',
    'three_points_made': '3PM',
    'shots': 'Shots',
    'goals': 'Goals',
    'saves': 'Goalie Saves',
    'clearances': 'Clearances',
    'goals_and_assists': 'Goals and Assists',
    'tackles': 'Tackles',
    'interceptions': 'Interceptions',
    'crosses': 'Crosses',
    'goals_against': 'Goal Against',
    'game_1_and_2_kills': 'MAPS 1-2 Kills',
    'map_1_and_2_kills': 'MAPS 1-2 Kills',
    'Headshots on Maps 1+2': 'MAPS 1-2 Headshots',
    'strokes': 'Strokes',
    'bogeys_or_worse': 'Bogeys or Worse',
    'birdies_or_better': 'Birdies or Better',
    'aces': 'Aces',
    'breakpoints_won': 'Break Points Won',
    'games_won': 'Games Won',
    'games_lost': 'Games Lost',
    'total_games': 'Total Games',
    'double_faults': 'Double Faults',
    'service_games_lost': 'Service Games Lost',
    'Fastest Laps': 'Fastest Laps',
    'position': 'position',
    'passing_yds': 'Pass Yards',
    'rushing_yds': 'Rush Yards',
    'receiving_yds': 'Receiving Yards' ,
    'fantasy_points': 'Hitter Fantasy Score',
    'strikeouts': 'Pitcher Strikeouts',
    'hits_runs_rbis': 'PTS',
    'singles':'Singles',
    'total_bases': 'Total Bases',
    'walks': 'Walks Allowed',
    'hits': 'Hits Allowed',
                }

In [50]:
df = df.replace(syntax)

In [51]:
df.loc[df['League'] == 'ESPORTS']

,Line,Stat,Player,League,Team,Date
285,45.5,kills_on_maps_1_2_3,CLAYSTER,ESPORTS,Vegas Legion,2023-05-25
286,22.0,map_1_kills,CLAYSTER,ESPORTS,Vegas Legion,2023-05-25
287,18.5,kills_on_map_3,CLAYSTER,ESPORTS,Vegas Legion,2023-05-25
288,46.5,kills_on_maps_1_2_3,TEMP,ESPORTS,Vegas Legion,2023-05-25
289,19.5,kills_on_map_3,TEMP,ESPORTS,Vegas Legion,2023-05-25
...,...,...,...,...,...,...
557,5.5,MAPS 1-2 Kills,Stinger,ESPORTS,beastcoast,2023-05-25
559,11.5,MAPS 1-2 Kills,SLATEM$,ESPORTS,Thunder Awaken,2023-05-25
560,9.5,MAPS 1-2 Kills,ILICH-,ESPORTS,Thunder Awaken,2023-05-25
561,5.5,MAPS 1-2 Kills,N1ght,ESPORTS,Thunder Awaken,2023-05-25


In [52]:
df.loc[df.Player=='Krimbo']

,Line,Stat,Player,League,Team,Date


In [53]:
today = dt.date.today()

In [54]:
str(today)

'2023-05-24'

In [55]:
df.to_csv(f'Lines/doglines{today.month}_{today.day}')

In [56]:
df.loc[df.League=='ESPORTS']

,Line,Stat,Player,League,Team,Date
285,45.5,kills_on_maps_1_2_3,CLAYSTER,ESPORTS,Vegas Legion,2023-05-25
286,22.0,map_1_kills,CLAYSTER,ESPORTS,Vegas Legion,2023-05-25
287,18.5,kills_on_map_3,CLAYSTER,ESPORTS,Vegas Legion,2023-05-25
288,46.5,kills_on_maps_1_2_3,TEMP,ESPORTS,Vegas Legion,2023-05-25
289,19.5,kills_on_map_3,TEMP,ESPORTS,Vegas Legion,2023-05-25
...,...,...,...,...,...,...
557,5.5,MAPS 1-2 Kills,Stinger,ESPORTS,beastcoast,2023-05-25
559,11.5,MAPS 1-2 Kills,SLATEM$,ESPORTS,Thunder Awaken,2023-05-25
560,9.5,MAPS 1-2 Kills,ILICH-,ESPORTS,Thunder Awaken,2023-05-25
561,5.5,MAPS 1-2 Kills,N1ght,ESPORTS,Thunder Awaken,2023-05-25
